In [ ]:
# 필요한 모듈 설치
!pip3 install -r ./requirement.txt

In [8]:
# 모듈 로드
import hnswlib
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# HNSW Class 구현
class HNDSWNearestNeighbor:
    def __init__(self, vectors, texts, space="cosine", ef_construction=200, M=16):
        self.texts = texts
        self.d = vectors.shape[1]
        self.index = hnswlib.Index(space=space, dim=self.d)
        num_elements = len(vectors)
        self.index.init_index(max_elements=num_elements, ef_construction=ef_construction, M=M)
        for i, vector in enumerate(vectors):
            self.index.add_items(vector, i)
    
    def search(self, query_vector, top_k=1):
        labels, distances = self.index.knn_query(query_vector, k=top_k)
        similarities = [1- d for d in distances[0]]
        similar_texts = [self.texts[i] for i in labels[0]]
        return similar_texts, similarities

In [7]:
# 테스트 코드 - Vector DB 구축
model = SentenceTransformer("jhgan/ko-sbert-sts")

text_list = [
    "원숭이가 노래를 한다.",
    "배가 바다를 떠나 원대한 여정을 시작했다.",
    "그는 자신이 벌레만도 못한 취급을 받을 것이라곤 생각할 수 없었다."
]
text_vectors = model.encode(text_list, normalize_embeddings=True)
vector_db = HNDSWNearestNeighbor(text_vectors, text_list, space="cosine", ef_construction=200, M=16)

In [9]:
# 테스트 코드 - VectorDB 테스트
query_text = "배가 바다를 떠나 원대한 여정을 시작함."
query_vector = model.encode(query_text, normalize_embeddings=True)
text, sim = vector_db.search(query_vector, 1)
print(f"유사 문장 : {text[0]}\t유사도 : {sim[0] : .4f}")
print(f"SBERT 유사도 : {float(util.cos_sim(model.encode(text), model.encode(query_text))) : .4f}") # Annoy 유사도와 동일해야함!

유사 문장 : 배가 바다를 떠나 원대한 여정을 시작했다.	유사도 :  0.9857
SBERT 유사도 :  0.9857
